<h2 class="section-heading">The Problem</h2>

Single neuron has limited computational power and hence we need a way to build a network of neurons to make a more complex model. In this post we will look into how to construct a neural network and try to solve the handwritten digit recognition problem. The goal is to decide which digit it represents when given a new image. 

<h2 class="section-heading">Understanding the Data</h2>

We'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Luckily, [John Myles White](https://github.com/johnmyleswhite/MNIST.jl) has already created a package to import this dataset in Julia. The MNIST dataset provides a training set of 60,000 handwritten digits and a test set of 10,000 handwritten digits. Each of the image has a size of 28×28 pixels. ![MNIST](files/img/MNIST_digits.png)

In [1]:
#Pkg.update();
Pkg.clone("https://github.com/johnmyleswhite/MNIST.jl.git");
Pkg.add("PyPlot")
#Pkg.installed();

INFO: Cloning MNIST from https://github.com/johnmyleswhite/MNIST.jl.git
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package database updated
INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of PyPlot
INFO: Use `Pkg.update()` to get the latest versions of your packages


For plotting, PyPlot is a good option. It provides a Julia interface to the Matplotlib plotting library from Python.

In [1]:
using MNIST
using PyPlot

In [2]:
# ===================
# load training data
# ===================
X,y = traindata();
X=X';
m = size(X, 1); # number of rows

60000

 in depwarn at ./deprecated.jl:63
 in int at deprecated.jl:49
 in anonymous at /home/vagrant/.julia/v0.4/MNIST/src/MNIST.jl:23
 in open at iostream.jl:114
 in traindata at /home/vagrant/.julia/v0.4/MNIST/src/MNIST.jl:58
 in include_string at loading.jl:153
 in execute_request_0x535c5df2 at /home/vagrant/.julia/v0.3/IJulia/src/execute_request.jl:157
 in eventloop at /home/vagrant/.julia/v0.3/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading In[2], in expression starting on line 4


<h2 class="section-heading">Training a model</h2>

We want to train a neural network with one input layer, one hidden layer and one output layer to recognize handwritten digits. Since the dataset contains 28×28 pixel images, our neural network will have $28*28=784$ input neurons, a variable number of hidden neurons and $10$ output neurons.

![2-layer-neuralNetwork](files/img/300px-Colored_neural_network.svg.png)

In [3]:
# ======================
# network setup
# ======================
inputLayerSize = size(X,2); # number of input features
hiddenLayerSize = 25; # variable
outputLayerSize = 10; # number of output classes

# since we are doing multiclass classification: more than one output neurons
# representing each output as an array of size of the output layer
eyeY = eye(outputLayerSize);
intY = [convert(Int64,i)+1 for i in y];
Y = Array(Int64,length(y),outputLayerSize);
for i = 1:m
    Y[i,:] = eyeY[intY[i],:];
end

# randomly initializing weights
epsilon_init = 0.12;
# including one bias neuron in input layer
# weights for the links connecting input layer to the hidden layer
Theta1 = rand(hiddenLayerSize, inputLayerSize+1)* 2 * epsilon_init - epsilon_init;
# including one bias neuron in hidden layer
# weights for the links connecting hidden layer to the output layer
Theta2 = rand(outputLayerSize, hiddenLayerSize+1)* 2 * epsilon_init - epsilon_init;

The appraoch to train a neural network is similar to what we have discussed in the [neuron post](http://lakshgupta.github.io/2015/05/21/ArtificialNeuron/).  But since now we have a network of neurons, the way we follow the steps is a bit different. We'll use the [backpropagation algorithm](http://www.cs.toronto.edu/~hinton/absps/naturebp.pdf). 

>Backpropagation works by approximating the non-linear relationship between the input and the output by adjusting the weight values internally. 
The operations of the Backpropagation neural networks can be divided into two steps: feedforward and Backpropagation. In the feedforward step, an input pattern is applied to the input layer and its effect propagates, layer by layer, through the network until an output is produced. The network's actual output value is then compared to the expected output, and an error signal is computed for each of the output nodes. Since all the hidden nodes have, to some degree, contributed to the errors evident in the output layer, the output error signals are transmitted backwards from the output layer to each node in the hidden layer that immediately contributed to the output layer. This process is then repeated, layer by layer, until each node in the network has received an error signal that describes its relative contribution to the overall error.
Once the error signal for each node has been determined, the errors are then used by the nodes to update the values for each connection weights until the network converges to a state that allows all the training patterns to be encoded. 
<p align="right">- [www.cse.unsw.edu.au](http://www.cse.unsw.edu.au/~cs9417ml/MLP2/BackPropagation.html)</p>

We'll discuss more about the backpropagation algorithm later but first let's collect the simple tools which are required for training a neural network.

<h4 class="section-heading">Activation Function: $g$</h4>

Considering [linear regression](http://lakshgupta.github.io/2015/05/27/LinearRegression/), using a linear activation function does not give us much advantage here. Linear function applied to a linear function is itself a linear function, and hence both the functions can be replaced by a single linear function. Moreover real world problems are generally more complex. A linear activation function may not be a good fit for the dataset we have. Therefore if the data we wish to model is non-linear then we need to account for that in our model. Sigmoid activation function is one of the reasonably good non-linear activation functions which we could use in our neural network.

$$sigmoid(z) = 1/(1 + e^{-z})$$

![sigmoid](files/img/400px-SigmoidFunction.png)

In [4]:
# ==============================================
# activation function: computes the sigmoid of z 
# z is the weighted sum of inputs
# ==============================================
function sigmoid(z)
    g = 1.0 ./ (1.0 + exp(-z));
    return g;
end

sigmoid (generic function with 1 method)

<h4 class="section-heading">Cost Function: $J$</h4>

The “squared error” cost function used for linear regression is not suitable for neural networks, because the non-linear nature of activation function is said to produce a non-convex plot of the cost function, i.e. with many local minima. For further explanation refer to this [thread](https://class.coursera.org/ml-005/forum/thread?thread_id=2773).

Therefore we will use the cross-entropy cost function. 
>the cross-entropy is positive, and tends toward zero as the neuron gets better at computing the desired output, y, for all training inputs, x.
<p align="right">- [Michael Nielsen](http://neuralnetworksanddeeplearning.com/chap3.html)</p>

Take a look at this [paper](https://www-i6.informatik.rwth-aachen.de/publications/download/861/GolikPavelDoetschPatrickNeyHermann--Cross-Entropyvs.SquaredErrorTrainingaTheoreticalExperimentalComparison--2013.pdf) to have a comparison between cross entropy cost function and squared error cost function. So considering:
$$J(\theta) = \frac{1}{m}(\sum_{i=1}^{m}cost(h_{\theta}(x^{(i)}),y^{(i)}))$$
where:
- $h_{\theta}(x^{(i)})$ is the predicted value (hypothesis)
- $y^{(i)}$ is the actual value (truth), and
\begin{eqnarray}
cost(h_{\theta}(x^{(i)}),y^{(i)})&=&\left\{
\begin{array}{l l}      
    -\log(h_{\theta}(x^{(i)}))   &   \mathrm{if} \: y=1 \\
    -\log(1-h_{\theta}(x^{(i)})) &  \mathrm{if}  \: y=0
\end{array}\right.,  \: h_{\theta}(x^{(i)})\in(0,1) \\ \nonumber
&=& - y^{(i)}\log{h_{\theta}(x^{(i)})} - (1-y^{(i)})\log(1-h_{\theta}(x^{(i)}))
\end{eqnarray}

Hence our cost function becomes:
$$J(\theta) = -\frac{1}{m}[\sum_{i=1}^{m} ( y^{(i)}\log{h_{\theta}(x^{(i)})} + (1-y^{(i)})\log({1-h_{\theta}(x^{(i)}))})]$$

We don't sum over the bias terms hence starting at 1 for the summation. But this cost function will work if we have a single neuron in the output layer. Let's generalize this cost function so that we could use it for $k$ neurons in the output layer. 
$$J(\theta) = -\frac{1}{m}\sum_{i=1}^{m} \sum_{i=1}^{k}[ y^{(i)}_k\log{(h_{\theta}(x^{(i)})_k)} + (1-y^{(i)}_k)\log({1-(h_{\theta}(x^{(i)}))_k)}]$$



<h4 class="section-heading">Regularization: $L^2$</h4>

Most regularization approaches add a parameter norm penalty $\Omega(\theta)$ to the loss function $J$ to achieve better generalization of the model. In case of $L^2$ regularization, also known as weight decay, the penalty is equal to the sum of the square of all of the weight vectors.

$$\Omega(\theta) = \frac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}((\theta_{ji}^l)^2)$$ 

where 
- $\lambda>0$, is known as the regularization parameter
- $m$ is the size of our training set
- $L$ in the equation is the layer number
- $s$ is the neuron unit in the corresponding layer

>Regularizers work by trading increased bias for reduced variance. An effective regularizer is one that makes a proﬁtable trade, that is it reduces variance signiﬁcantly while not overly increasing the bias.
<p align="right">- [Yoshua Bengio, Ian Goodfellow and Aaron Courville](http://www.iro.umontreal.ca/~bengioy/dlbook/regularization.html)</p>

The Wikipedia has a descent article on the [bias-variance tradeoff](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff).

In [5]:
# weight regularization parameter
lambda = 3; 

# ===============================================
# cross entropy cost function with regularizarion
# ===============================================
function costFunction(prediction, truth)
    cost = zeros(m,1); # cost for each input
    for i=1:m
        cost[i,:] = (-truth[i,:]*log(prediction[i,:]')) - ((1-truth[i,:])*log(1-prediction[i,:]'));
    end

    # regularization term
    regularization = (lambda/(2*m))*(sum(sum(Theta1[:,2:end].^2)) + sum(sum(Theta2[:,2:end].^2)));

    return (1/m)*sum(cost) + regularization; # regularized cost
end

costFunction (generic function with 1 method)

<h4 class="section-heading">Backpropagation</h4>

Despite the name, [backpropagation algorithm](http://ufldl.stanford.edu/wiki/index.php/Backpropagation_Algorithm) consist of two phases:
- Feedforward 
- Backpropagation

The feedforward process is the same process we have been following in the previous posts. Using the feedforward process we calculate the weighted sum of inputs and apply the activation function to get an output as we move from layer to layer. In the end we come up with a output activation which could have some error as compared to the actual values. To have the output as close as possible to the actual values we use the backpropagation process to tune the weights.

Backpropagation is a way of computing gradients of expressions through recursive application of chain rule. We start from the output layer and go backwards calulating the gradient on the activations for each layer till the first hidden layer.

>From these gradients, which can be interpreted as an indication of how each layer’s output should change to reduce error, one can obtain the gradient on the parameters of each layer. The gradients on weights and biases can be immediately used as part of a stochastic gradient update (performing the update right after the gradients havebeen computed) or used with other gradient-based optimization methods.
<p align="right">- [Yoshua Bengio, Ian Goodfellow and Aaron Courville](http://www.iro.umontreal.ca/~bengioy/dlbook/regularization.html)</p>

Since we are learning/tuning weights ($\theta$), we want to evaluate: $\dfrac{\partial E}{\partial \theta_{ij}^{l}}$

here,
$$ \theta_{ij}^{l} =
\begin{cases}
1 \ge l \ge L,  & \text{layer} \\
0 \ge i \ge d^{(l-1)}, & \text{input} \\
1 \ge i \ge d^{(l)}, & \text{output}
\end{cases}
$$

Using the chain rule, we can solve the above partial derivative as:
$$\dfrac{\partial E}{\partial \theta_{ij}^{l}} = \underbrace{\dfrac{\partial E}{\partial s_{j}^{l}}}_{1} \underbrace{\dfrac{\partial s_{j}^{l}}{\partial \theta_{ij}^{l}}}_{2}$$

here, $s$ represents the input signal to a neuron which is the weighted sum of the inputs or in other words weighted sum of the outputs from the previous layer's neurons. Hence $(2)$ becomes:

$$\dfrac{\partial s_{j}^{l}}{\partial \theta_{ij}^{l}} = x^{(l-1)}$$

where, $x$ is the value from applying activation function $g$ to $s$. Now let's look at $(1)$ and represent it as $\delta$. Since we start backpropagation from the last output layer, we can numerically calculate it as:
$$\delta_{j}^{l} = \dfrac{\partial E}{\partial s_{j}^{l}} = error(x^{l},y)$$

$error$ could be any function to tell us how much far off we are from the actual value. For the MNIST problem, we'll simply subtract the predicted values from the actual values. 

For the hidden layers, we can break down $\delta$ as:
\begin{eqnarray}
\delta_{i}^{l-1}&=& \dfrac{\partial E}{\partial s_{i}^{l-1}} \\
&=& \sum_{j=1}^{d^{(l)}} (\dfrac{\partial E}{\partial s_{j}^{l}}) (\dfrac{\partial s_{j}^{l}}{\partial x_{i}^{l-1}}) (\dfrac{\partial x_{i}^{l-1}}{\partial s_{i}^{l-1}}) \\
&=& \sum_{j=1}^{d^{(l)}} [(\delta_{j}^{l}) (\theta_{ij}^{l}) (g'(s_i^{l-1}))]
\end{eqnarray}

We have now all the components for $\dfrac{\partial E}{\partial \theta_{ij}^{l}}$, hence we can update the weights as:
$$\theta_{ij}^{(l)} \leftarrow \theta_{ij}^{(l)} - \alpha (\delta_{j}^{l} x_{i}^{l-1})$$

In [6]:
# computes the gradient of the sigmoid function evaluated at z
function sigmoidGradient(z)
  return sigmoid(z).*(1-sigmoid(z));
end

sigmoidGradient (generic function with 1 method)

In [21]:
# learning rate
alpha = 0.1;
# number of iterations
epoch = 100;
# cost per epoch
J = zeros(epoch,1);
# ====================================================================
# Train the neural network using feedforward-backpropagation algorithm
# ====================================================================
for i = 1:epoch
    # ===================
    # Feedforward process
    # ===================
    # input layer
    # add one bias element
    x1 = [ones(size(X,1),1) X];

    # hidden layer
    s2 = x1*Theta1';
    x2 = sigmoid(s2);
    # add one bias element
    x2 = [ones(size(x2,1),1) x2];

    # output layer
    s3 = x2*Theta2';
    x3 = sigmoid(s3);
    
    # ===========================
    # find cost of the new output
    # ===========================
    J[i,:] = costFunction(x3, Y);
        
    # =======================
    # Backpropagation process
    # =======================
    inputLayerRegularizedGradient = zeros(size(Theta1));
    hiddenLayerRegularizedGradient = zeros(size(Theta2));

    # delta for output layer
    delta3 = x3 - Y;
    
    # gradient for hidden layer
    theta2_regularization = ((lambda/m)*Theta2);
    theta2_regularization[:,1] = 0;
    hiddenLayerRegularizedGradient = (delta3'*x2)/m #+ theta2_regularization;
    # delta for hidden layer
    delta2 = (delta3*Theta2[:, 2:end]).*sigmoidGradient(s2) ;

    # gradient for input layer
    theta1_regularization = ((lambda/m)*Theta1);
    theta1_regularization[:,1] = 0;
    inputLayerRegularizedGradient = (delta2'*x1)/m #+ theta1_regularization;
    # there is no delta term for the input layer
        
    # adjust the weights (thetas)
    Theta1 = Theta1 - alpha* inputLayerRegularizedGradient;
    Theta2 = Theta2 - alpha* hiddenLayerRegularizedGradient;
    
end

If our implementation is correct, the cost of the predicted output after each iteration should drop. I'll cover another method (gradient check) in another post to validate our implementation. But for now let's check by plotting the cost per iteration.

In [8]:
# plot the cost per iteration
plot(1:length(J), J)
xlabel("Iterations")
ylabel("Cost")
grid("on")

<h2 class="section-heading">Prediction and Accuracy</h2>

After training the model we'll check how well our model has learned all the weights to make a prediction. So we'll evaluate the performance on the train dataset first. We do it by following a way similar to the feedforward process. Consider that we now do not have randonly initialized weights but rather obtained after going through the backpropagation algorithm.

In [9]:
##########################################################################
# outputs the predicted label of X given the
# trained weights of a neural network (Theta1, Theta2)
# Similar to feedforward process.
##########################################################################
function predict(Theta1, Theta2, X)
    m = size(X, 1); # size of the data
    p = zeros(size(X, 1), 1); # to save our prediction
    h1 = sigmoid([ones(m, 1) X] * Theta1'); # hidded layer output
    h2 = sigmoid([ones(m, 1) h1] * Theta2'); # output layer
    # find the index with the max value in the array of size 10
    # subtract 1 from the index since we are using 1 to 
    # represent 0, 2 for 3 and so on (while calculating yInter)
    for i=1:m
        p[i,:] = indmax(h2[i,:])-1;
      end
 return p;
end

predict (generic function with 1 method)

In [22]:
# make prediction
pred = predict(Theta1, Theta2, X);

In [11]:
###############################################
# calculate the accuracy of the prediction
###############################################
function accuracy(truth, prediction)
    m = length(truth);
    if m!=length(prediction)
        error("truth and prediction length mismatch");
    end
    
    # calculate the % of predicted values
    # matching the actual values
    sum =0;
    for i=1:m
        if truth[i,:] == pred[i,:]
            sum = sum +1;
        end
    end
  return (sum/m)*100;
end

accuracy (generic function with 1 method)

In [23]:
# calculate accuracy
println("train accuracy: ", accuracy(y, pred));

train accuracy: 9.93


After calculating the accuracy on the train dataset, let's check the accuracy on the test dataset to be sure that we did not overfit the data. If there is too much difference then we might have to tune some parameters.

In [13]:
# ===============
# load test data
# ===============
XTest,yTest = testdata();
XTest=XTest';

In [19]:
# make prediction
predTest = predict(Theta1, Theta2, XTest);

In [20]:
# calculate accuracy
println("test accuracy: ", accuracy(yTest, predTest));

test accuracy: 9.66


That's all folks! Now our model can make the prediction on any new handwritten digit in a similar way as we made the prediction on the test dataset. There are different ways to improve the model but I'll leave them for the coming posts.

<h2 class="section-heading">References:</h2>

- [Deep Learning in Neural Networks: An Overview](http://arxiv.org/abs/1404.7828)
- [Learning representations by back-propagating errors](http://www.cs.toronto.edu/~hinton/absps/naturebp.pdf)
- [Coursera Machine Learning](https://class.coursera.org/ml-005)
- [CS 231n](http://cs231n.github.io/optimization-2/)
- [Deep Learning](http://www.iro.umontreal.ca/~bengioy/dlbook/mlp.html)
- [UDFL](http://ufldl.stanford.edu/wiki/index.php/Backpropagation_Algorithm)
- [Learning from Data](http://work.caltech.edu/slides/slides10.pdf)